# This notebook will be mainly used for the capstone project

In [5]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Scrapping the wikipedia page of Toronto

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url).read()
bs = BeautifulSoup(html)
table = bs.findAll('table')

In [8]:
rows = table[0].findAll('tr')

In [9]:
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])

In [10]:
for r in rows:
    elements = r.findAll('td')
    toadd = []
    for e in elements:
        toadd.append(e.text)
    if len(toadd) == 3:
        df = df.append({'PostalCode': toadd[0][0:-1], 'Borough': toadd[1][0:-1], 'Neighborhood': toadd[2][0:-1]}, ignore_index=True)

In [11]:
df = df[df['Borough'] != 'Not assigned']
df = df.reset_index(drop = True)

In [12]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [13]:
df.shape

(103, 3)

# Get Latitude and the Longitude of Neighborhoods

In [20]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")

In [21]:
lat = []
lon = []
for index, row in df.iterrows():
    location = geolocator.geocode('{}, {}, Toronto, Ontario'.format(row['Neighborhood'], row['Borough']))
    if location is None:
        location = geolocator.geocode('{}, Toronto, Ontario'.format(row['Borough']))
    lat.append(location.latitude)
    lon.append(location.longitude)

In [22]:
lat = np.asarray(lat, dtype = float)
lon = np.asarray(lon, dtype = float)
df['latitude'] = lat
df['longitude'] = lon

In [23]:
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.656322,-79.380916
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.656322,-79.380916
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.643556,-79.565633
99,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.624790,-79.393492
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.643556,-79.565633


### Notice that some Borough have the same latitude and longitude. Hence, I used the csv file provided

In [24]:
coor = pd.read_csv('Geospatial_Coordinates.csv')
coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
lat = []
lon = []
for index, row in df.iterrows():
    location = geolocator.geocode('{}, {}, Toronto, Ontario'.format(row['Neighborhood'], row['Borough']))
    if location is None:
        latlon = coor[coor['Postal Code'] == row['PostalCode']]
        lat.append(float(latlon['Latitude']))
        lon.append(float(latlon['Longitude']))
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

df['latitude'] = np.asarray(lat, dtype = float)
df['longitude'] = np.asarray(lon, dtype = float)

In [27]:
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Explore and cluster the neighborhoods in Toronto

In [28]:
CLIENT_ID = 'Write ID' # your Foursquare ID
CLIENT_SECRET = 'Write Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZELAP14ONQQOXRVT0ZMJ13J24LMXDQEDFOX0DFYE12543Y00
CLIENT_SECRET:AMNCVPB4C1PEX452LUBIUJUB01WRH420OQ00UG12HD1QBXXT


In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [32]:
import requests

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [42]:
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.758800,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.758800,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.758800,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
3,Parkwoods,43.758800,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
4,Parkwoods,43.758800,-79.320197,Pizza Pizza,43.760231,-79.325666,Pizza Place
...,...,...,...,...,...,...,...
2855,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,RONA,43.629393,-79.518320,Hardware Store
2856,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2857,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2858,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [44]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.076923,0.0,0.0,0.000000,0.00
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.016393,0.0,0.0,0.0,0.049180,0.0,0.0,0.016393,0.00
94,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.016393,0.0,0.0,0.0,0.049180,0.0,0.0,0.016393,0.00
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.045455,0.0,0.0,0.000000,0.00
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00


In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Restaurant,Shopping Mall,Vietnamese Restaurant,Coffee Shop,Food Court,Hong Kong Restaurant,Korean Restaurant,Asian Restaurant,Train Station
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Dance Studio,Pub,Pharmacy,Gym,Deli / Bodega,Electronics Store,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Frozen Yogurt Shop,Pizza Place,Deli / Bodega,Restaurant,Mobile Phone Shop,Diner,Middle Eastern Restaurant
3,Bayview Village,Bank,Gas Station,Breakfast Spot,Pizza Place,Persian Restaurant,Outdoor Supply Store,Fast Food Restaurant,Sporting Goods Shop,Sandwich Place,Fish Market
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Grocery Store,Butcher,Café,Pub,Pizza Place,Women's Store


In [49]:
from sklearn.cluster import KMeans

In [51]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 3, 4, 0, 0, 0, 3, 1, 3, 0, 3, 0,
       0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0,
       2, 3, 0, 0, 4, 0, 0, 0, 3, 0], dtype=int32)

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,0.0,Electronics Store,Caribbean Restaurant,Pharmacy,Discount Store,Pizza Place,Shopping Mall,Coffee Shop,Liquor Store,Convenience Store,Bank
1,M4A,North York,Victoria Village,43.732658,-79.311189,0.0,Thai Restaurant,Spa,Asian Restaurant,Middle Eastern Restaurant,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Historic Site,Restaurant,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Women's Store,Coffee Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Event Space,Accessories Store,Vietnamese Restaurant,Community Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Diner,Yoga Studio,Sandwich Place,Bank,Bar,Sushi Restaurant,Beer Bar,Mexican Restaurant,Distribution Center


In [56]:
import folium

In [58]:
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

In [60]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [73]:
Toronto_merged = Toronto_merged.dropna()

In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['latitude'], Toronto_merged['longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [77]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Electronics Store,Caribbean Restaurant,Pharmacy,Discount Store,Pizza Place,Shopping Mall,Coffee Shop,Liquor Store,Convenience Store,Bank
1,North York,0.0,Thai Restaurant,Spa,Asian Restaurant,Middle Eastern Restaurant,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,0.0,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Historic Site,Restaurant,Performing Arts Venue
3,North York,0.0,Clothing Store,Women's Store,Coffee Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Event Space,Accessories Store,Vietnamese Restaurant,Community Center
4,Downtown Toronto,0.0,Coffee Shop,Diner,Yoga Studio,Sandwich Place,Bank,Bar,Sushi Restaurant,Beer Bar,Mexican Restaurant,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,0.0,Coffee Shop,Café,Bakery,Italian Restaurant,Pizza Place,Pub,Park,Restaurant,Snack Place,Butcher
97,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Steakhouse,Deli / Bodega,Seafood Restaurant,Asian Restaurant
99,Downtown Toronto,0.0,Coffee Shop,Park,Diner,Burrito Place,Sushi Restaurant,Beer Bar,Smoothie Shop,Ethiopian Restaurant,Martial Arts Dojo,Bubble Tea Shop
100,East Toronto,0.0,Light Rail Station,Spa,Restaurant,Skate Park,Auto Workshop,Recording Studio,Burrito Place,Fast Food Restaurant,Garden Center,Farmers Market


### Cluster 2

In [78]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,1.0,Fabric Shop,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
101,Etobicoke,1.0,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Department Store


### Cluster 3

In [79]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Jewelry Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Department Store


### Cluster 4

In [80]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,3.0,Playground,Women's Store,Donut Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
12,Scarborough,3.0,Bar,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Department Store
14,East York,3.0,Skating Rink,Dance Studio,Athletics & Sports,Bus Stop,Pharmacy,Park,Comfort Food Restaurant,Comedy Club,Event Space,Ethiopian Restaurant
16,York,3.0,Trail,Field,Hockey Arena,Park,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
23,East York,3.0,Sandwich Place,Park,Japanese Restaurant,Convenience Store,Women's Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center
44,Scarborough,3.0,Bakery,Bus Line,Intersection,Bus Station,Ice Cream Shop,Soccer Field,Metro Station,Park,Electronics Store,Eastern European Restaurant
49,North York,3.0,Deli / Bodega,Basketball Court,Bakery,Park,Construction & Landscaping,Trail,Discount Store,Distribution Center,Dog Run,Doner Restaurant
50,North York,3.0,Bakery,Park,Construction & Landscaping,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
68,Central Toronto,3.0,Jewelry Store,Park,Sushi Restaurant,Trail,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
70,Etobicoke,3.0,Pizza Place,Gas Station,Park,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doner Restaurant


### Cluster 5

In [81]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Scarborough,4.0,Park,Women's Store,Donut Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
21,York,4.0,Park,Women's Store,Pool,Donut Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
35,East York,4.0,Park,Convenience Store,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
52,North York,4.0,Park,Women's Store,Donut Shop,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
